In [30]:
from pymodules import sim_functions as sf
from pymodules.sim_functions import b2
import time
import json
from pymodules.ML_params import *

In [14]:
cell = "NMO_00892"

In [15]:
morph_d8 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth8.swc", spherical_soma=True)
morph_d13 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth13.swc", spherical_soma=True)
morph_d14 = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_depth14.swc", spherical_soma=True)
morph_full = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_full.swc", spherical_soma=True)

In [16]:
depths = ["depth8", "depth13", "depth14", "full"]

In [17]:
morph_soma = b2.spatialneuron.morphology.Morphology.from_swc_file(f"morphs/{cell}_somas.swc", spherical_soma=True)

In [18]:
α = 4.5 # scaling factor for the soma
β = 1.0 # scaling factor for the dendrites

In [19]:
neuron_soma = b2.SpatialNeuron(morphology=morph_soma, model=eqs, Cm=α*Cm, Ri=250*b2.ohm*b2.cm, method='exponential_euler')

In [20]:
neuron_soma.gL = α*gL0
neuron_soma.EL = EL0
Iin = 0.1*b2.nA
tmax = 500*b2.ms
v0 = EL0
gσ = sf.gin(neuron_soma, Iin, v0, tmax)
print(gσ)

Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
5.05465572 nS


In [21]:
len_σ = len(neuron_soma)
print(len_σ)

36


In [22]:
def gin_depths(morph_set, len_σ, gσ):
    gin = b2.zeros(len(morph_set))*b2.siemens
    
    ρ = b2.zeros(len(morph_set))
    l = b2.zeros(len(morph_set))
    d = b2.zeros(len(morph_set))*b2.um
    Iin = 0.1*b2.nA
    tmax = 500*b2.ms
    v0 = EL0
    for i,j in enumerate(morph_set):
        neuron = b2.SpatialNeuron(morphology=morph_set[i], model=eqs, Cm=α*Cm, Ri=250*b2.ohm*b2.cm, method='exponential_euler')
        neuron[0:len_σ].gL = α*gL0
        neuron[len_σ::].gL = β*gL0
        neuron.EL = EL0
        neuron.Cm[len_σ::] = 0.25*β*Cm # this makes the dendritic time constant τδ = 2.5 ms
        
        gin[i] = sf.gin(neuron, Iin, v0, tmax)
        Z0, ZL = sf.Z0L(neuron, Iin, v0, tmax)

        l[i] = sf.ct.l_equiv(Z0, ZL)
        d[i] = sf.ct.d_equiv(Z0, 250*b2.ohm*b2.cm, gL0, l[i])
        ρ[i] = (gin[i]-gσ)/gσ

    return gin, ρ, l, d

In [23]:
gset, ρset, lset, dset = gin_depths([morph_d8, morph_d13, morph_d14, morph_full], len_σ, gσ)

Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0.5 s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0.5 s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0.5 s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0. s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s
Starting simulation at t=0.5 s for a duration of 0.5 s
0.5 s (100%) simulated in < 1s


In [24]:
ρset

array([0.42352295, 1.28093027, 1.48602477, 1.92785077])

In [26]:
lset

array([0.27316844, 0.52186288, 0.58906686, 0.75972089])

In [27]:
dset

array([11.38691164, 10.54725957, 10.45398011, 10.26135203]) * umetre

In [28]:
jdict = {"depths": depths, "gs": gσ/b2.nS, "rhoinf": ρset.tolist(), "leff": lset.tolist(), "deff": (dset/b2.um).tolist()} 

In [31]:
with open(f"data/{cell}_cylinder_parameters.json", "w") as json_file:
    json.dump(jdict, json_file)